# 室内伝達関数のシミュレーションと畳み込み

In [62]:
import wave as wave
import pyroomacoustics as pa
import numpy as np
import os

In [63]:
np.random.seed(0)

In [64]:
clean_wave_files=["./dataset/cmu_us_aew_arctic/wav/arctic_a0001.wav", "./dataset/cmu_us_axb_arctic/wav/arctic_a0002.wav"]
for sound_file in clean_wave_files:
    if not os.path.exists(sound_file):
        print(f"{sound_file} is not found")

In [65]:
# 音源数
n_sources=len(clean_wave_files)

In [66]:
# ファイルの長さ
n_samples = 0
for clean_wave_file in clean_wave_files:
    with wave.open(clean_wave_file) as wav:
        if n_samples < wav.getnframes():
            n_samples = wav.getnframes()
print(n_samples)

62081


In [67]:
clean_data = np.zeros([n_sources, n_samples])

In [68]:
# 音源データの読み込み
s = 0
for clean_wave_file in clean_wave_files:
    wav = wave.open(clean_wave_file)
    if wav is None:
        raise FileExistsError(f"{clean_wave_file} is not found.")

    data = wav.readframes(wav.getnframes())
    data = np.frombuffer(data, dtype=np.int16)
    data = data/np.iinfo(np.int16).max
    clean_data[s, :wav.getnframes()] = data
    wav.close()
    s+=1

In [69]:
# シミュレーションのパラメータ設定
sample_rate = 16000
SNR=10
room_dim = np.r_[10.0, 10.0, 10.0] # 部屋の大きさ

# マイクロホンアレイを置く部屋の位置
mic_array_loc = room_dim / 2 * np.random.randn(3)*0.1
print(f"mic array pos: {mic_array_loc}")

# マイクロホンアレイ中心に対するマイクロホン配置
mic_alignments = np.array(
    [
        [-0.01, 0., 0.],
        [0.01, 0., 0.],
        [0.03, 0., 0.]
    ]
)

n_channels = np.shape(mic_alignments)[0]
R = mic_alignments.T + mic_array_loc[:, None]

print(R.T)

mic array pos: [0.88202617 0.2000786  0.48936899]
[[0.87202617 0.2000786  0.48936899]
 [0.89202617 0.2000786  0.48936899]
 [0.91202617 0.2000786  0.48936899]]


In [70]:
# 部屋を生成
room = pa.ShoeBox(room_dim, fs=sample_rate, max_order=17, absorption=0.35)
room.add_microphone_array(pa.MicrophoneArray(R, fs=room.fs))

In [71]:
# 音源の場所
doas = np.array(
    [
        [np.pi / 2., 0],
        [np.pi / 2., np.pi/2.]
    ]
)

In [72]:
# 音源とマイクロホンの距離
distance=1.
source_locations=np.zeros((3, doas.shape[0]), dtype=doas.dtype)
source_locations[0, :] = np.cos(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[1, :] = np.sin(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[2, :] = np.cos(doas[:, 0])

source_locations *= distance
source_locations += mic_array_loc[:, None]
print(source_locations.T)

[[1.88202617 0.2000786  0.48936899]
 [0.88202617 1.2000786  0.48936899]]


In [73]:
# 各音源をシミュレーションに追加する
for s in range(n_sources):
    clean_data[s] /= np.std(clean_data[s])
    room.add_source(source_locations[:, s], signal=clean_data[s])

In [74]:
room.simulate(snr=SNR)

# インパルス 応答の取得と残響時間の取得

In [75]:
inpulse_responses = room.rir
rt60 = pa.experimental.measure_rt60(inpulse_responses[0][0], fs=sample_rate)
print(f"残響時間: {rt60} sec")

残響時間: 0.348125 sec
